### **CARGA EN BASE DE DATOS: *MONGODB***

MongoDB es un sistema de base de datos NoSQL, orientado a documentos y de código abierto que, en lugar de guardar los datos en tablas, los guarda en estructuras de datos BSON, similar a JSON.

Se cargan en este sistema los cuatro archivos csv generados para futuros análisis de los mismos.

In [3]:

import pandas as pd
from pymongo import MongoClient


In [8]:
llam = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/llamadas_016.csv')
denu = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/denuncias_vg.csv')
vict = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/victimas_mortales_vg.csv')
camp = pd.read_csv('/Users/noeliarosonmartin/Ironhack/Project_ETL/data/data_clean/campañas_vg.csv')


In [4]:
client = MongoClient("mongodb://localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [5]:
# Creación de la base de datos:

db = client['impacto_vg']    

In [11]:
# Inserción de los datos:

db['llamadas'].insert_many(llam.to_dict(orient='records'))
db['denuncias'].insert_many(denu.to_dict(orient='records'))
db['victimas'].insert_many(vict.to_dict(orient='records'))
db['campañas'].insert_many(camp.to_dict(orient='records'))

InsertManyResult([ObjectId('65494598382a5666bab0542a'), ObjectId('65494598382a5666bab0542b'), ObjectId('65494598382a5666bab0542c'), ObjectId('65494598382a5666bab0542d'), ObjectId('65494598382a5666bab0542e'), ObjectId('65494598382a5666bab0542f'), ObjectId('65494598382a5666bab05430'), ObjectId('65494598382a5666bab05431'), ObjectId('65494598382a5666bab05432'), ObjectId('65494598382a5666bab05433')], acknowledged=True)

In [12]:
db.list_collection_names()

['campañas', 'llamadas', 'denuncias', 'victimas']

#### **¿EN QUÉ MES DE 2020 HUBO CAMPAÑA DE PREVENCIÓN?**

In [13]:

query = { 'Año': 2020}

select = {  'Año': 1,
            '_id': 0,
            'Mes': 1}

resultado = list(db['campañas'].find(query, select))

f'Hubo campaña en {resultado}'


"Hubo campaña en [{'Año': 2020, 'Mes': 11}]"


#### **¿QUÉ PROVINCIAS RECIBIERON MÁS LLAMADAS AL 016 EN LOS MESES SIGUIENTES A LA CAMPAÑA DE NOVIEMBRE 2020?**

In [14]:
query = {'Año': 2020,
         'Mes': {'$in': [11, 12]}}     
select = {  'Provincia': 1,
            '_id': 0,
            'Mes': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).sort('Numero_llamadas', -1).limit(5))

resultado

[{'Provincia': 'Madrid', 'Mes': 12, 'Numero_llamadas': 1115},
 {'Provincia': 'Madrid', 'Mes': 11, 'Numero_llamadas': 1087},
 {'Provincia': 'Barcelona', 'Mes': 12, 'Numero_llamadas': 517},
 {'Provincia': 'Barcelona', 'Mes': 11, 'Numero_llamadas': 469},
 {'Provincia': 'Valencia/València', 'Mes': 12, 'Numero_llamadas': 323}]

#### **¿Y SI LO COMPARAMOS CON LOS MESES ANTERIORES?**

Ejemplo: Sevilla

In [15]:
query = {'Año': 2020,
         'Mes': {'$in': [8, 9, 10, 11, 12]},
         'Provincia': 'Sevilla'}     
select = { '_id': 0,
            'Mes': 1,
            'Año': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).limit(10))

resultado

[{'Año': 2020, 'Mes': 8, 'Numero_llamadas': 296},
 {'Año': 2020, 'Mes': 9, 'Numero_llamadas': 278},
 {'Año': 2020, 'Mes': 10, 'Numero_llamadas': 207},
 {'Año': 2020, 'Mes': 11, 'Numero_llamadas': 213},
 {'Año': 2020, 'Mes': 12, 'Numero_llamadas': 232}]

Ejemplo: Barcelona

In [17]:
query = {'Año': 2020,
         'Mes': {'$in': [8, 9, 10, 11, 12]},
         'Provincia': 'Barcelona'}     
select = { '_id': 0,
            'Mes': 1,
            'Año': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).limit(10))

resultado

[{'Año': 2020, 'Mes': 8, 'Numero_llamadas': 709},
 {'Año': 2020, 'Mes': 9, 'Numero_llamadas': 606},
 {'Año': 2020, 'Mes': 10, 'Numero_llamadas': 502},
 {'Año': 2020, 'Mes': 11, 'Numero_llamadas': 469},
 {'Año': 2020, 'Mes': 12, 'Numero_llamadas': 517}]

Como observamos, en esas dos ciudades las llamadas bajaron tras la primera campaña. Vamos a observar esos dos ejemplos y vamos a comprobar las llamadas en esos meses del año siguiente. Pero primero comprobaremos cuántas campañas hubo durante el 2021.

In [37]:
query = { 'Año': 2021}

select = {'_id': 0,
          'Mes': 1}

resultado = list(db['campañas'].find(query, select))
Mes = resultado

f'Hubo {len(resultado)} campañas en los meses {Mes} de 2021'

"Hubo 6 campañas en los meses [{'Mes': 11}, {'Mes': 11}, {'Mes': 10}, {'Mes': 5}, {'Mes': 4}, {'Mes': 3}] de 2021"

In [12]:
query = {'Año': 2021,
         'Mes': {'$in': [8, 9, 10, 11, 12]},
         'Provincia': 'Sevilla'}     
select = { '_id': 0,
            'Mes': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).limit(10))

resultado

[{'Mes': 8, 'Numero_llamadas': 297},
 {'Mes': 9, 'Numero_llamadas': 273},
 {'Mes': 10, 'Numero_llamadas': 304},
 {'Mes': 11, 'Numero_llamadas': 253},
 {'Mes': 12, 'Numero_llamadas': 248}]

Se observa una pequeña subida en las llamadas en los meses de octubre, noviembre y diciembre de 2021, con respecto a 2020, en Sevilla.

In [11]:
query = {'Año': 2021,
         'Mes': {'$in': [8, 9, 10, 11, 12]},
         'Provincia': 'Barcelona'}     
select = { '_id': 0,
            'Mes': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).limit(10))

resultado

[{'Mes': 8, 'Numero_llamadas': 793},
 {'Mes': 9, 'Numero_llamadas': 729},
 {'Mes': 10, 'Numero_llamadas': 782},
 {'Mes': 11, 'Numero_llamadas': 757},
 {'Mes': 12, 'Numero_llamadas': 671}]

En Barcelona se observa un incremento generalizado en todos esos meses con respecto al año anterior, si bien es cierto que en los meses de noviembre y diciembre el aumento parece mayor.

Centrándonos en la ciudad de Sevilla, vamos a observar cuál es el mes con más llamadas en todos los años:

In [17]:
query = {'Provincia': 'Sevilla'}     
select = { '_id': 0,
            'Mes': 1,
            'Año': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).sort('Numero_llamadas', -1).limit(3))

resultado

[{'Año': 2020, 'Mes': 6, 'Numero_llamadas': 405},
 {'Año': 2021, 'Mes': 6, 'Numero_llamadas': 394},
 {'Año': 2020, 'Mes': 5, 'Numero_llamadas': 384}]

Y ahora, el mes con menos llamadas en todos los años:

In [18]:
query = {'Provincia': 'Sevilla'}     
select = { '_id': 0,
            'Mes': 1,
            'Año': 1,
            'Numero_llamadas': 1}

resultado = list(db['llamadas'].find(query, select).sort('Numero_llamadas').limit(3))

resultado

[{'Año': 2021, 'Mes': 2, 'Numero_llamadas': 163},
 {'Año': 2020, 'Mes': 10, 'Numero_llamadas': 207},
 {'Año': 2020, 'Mes': 11, 'Numero_llamadas': 213}]

#### **¿CUÁL FUE EL TRIMESTRE CON MÁS DENUNCIAS EN LA CIUDAD DE SEVILLA?**

In [36]:
query = {'Provincia': 'Sevilla'}     
select = { '_id': 0,
            'Trimestre': 1,
            'Año': 1,
            'Numero_denuncias': 1}

resultado = list(db['denuncias'].find(query, select).sort('Numero_denuncias', -1).limit(1))

resultado

[{'Año': 2022, 'Trimestre': 3, 'Numero_denuncias': 1980}]